In [1]:
import logging
from kiteconnect import KiteConnect
logging.basicConfig(level=logging.ERROR)
import threading
kite = KiteConnect(api_key="w19o0chuo929jxkp")
import joblib
kite = joblib.load('kitefile.p')
kite.login_url()

'https://kite.trade/connect/login?api_key=w19o0chuo929jxkp&v=3'

In [ ]:
mytoken = input('Enter Token Here : ')

data = kite.generate_session(mytoken, 
                             api_secret="gsw8ps17ex7lf3cuji4prfnwb4vlyr4y")
kite.set_access_token(data["access_token"])
import joblib
joblib.dump(kite,'kitefile.p')

In [ ]:
import act

In [ ]:
kite.orders()[-1]

In [ ]:
kite.positions()['net']

In [ ]:
kite.holdings()

In [ ]:
kite.margins(segment = 'equity')['available']['live_balance']

In [2]:
#####################
pivot_price = 69.6950
#####################
fullquantity = 90
per_order_quant = 10
#####################
initial_difference = 0.0000
difference = 0.0225
order_type = 'MIS' # MIS / NRML
symbol_ip = 'JPYINR20OCTFUT'
inst_token = 690435 # 690691(USD) / 690435(JPY)
#####################

stopbuy = False
stopsell = False

base=0.0025
max_no_of_order = int(fullquantity / per_order_quant)
total_parts = 2*max_no_of_order

print()
print('pivot_price : '+str(pivot_price))
print()

def calc_pivots():
    for i in range(1,max_no_of_order+1):
        globals()['p'+str(i)+'b'] = pivot_price - (initial_difference + i* difference)
        globals()['p'+str(i)+'b'] = round(base * round(globals()['p'+str(i)+'b'] /base),4)
        print('p'+str(i)+'b : '+str(globals()['p'+str(i)+'b']))

    print()
    for i in range(1,max_no_of_order+1):
        globals()['p'+str(i)+'s'] = pivot_price + (initial_difference + i* difference)
        globals()['p'+str(i)+'s'] = round(base * round(globals()['p'+str(i)+'s'] /base),4)
        print('p'+str(i)+'s : '+str(globals()['p'+str(i)+'s']))

    print()
    for i in range(1,max_no_of_order+1):
        globals()['p'+str(i)+'q'] = per_order_quant*i
        print('p'+str(i)+'q : '+str(globals()['p'+str(i)+'q']))

calc_pivots()


pivot_price : 69.695

p1b : 69.6725
p2b : 69.65
p3b : 69.6275
p4b : 69.605
p5b : 69.5825
p6b : 69.56
p7b : 69.5375
p8b : 69.515
p9b : 69.4925

p1s : 69.7175
p2s : 69.74
p3s : 69.7625
p4s : 69.785
p5s : 69.8075
p6s : 69.83
p7s : 69.8525
p8s : 69.875
p9s : 69.8975

p1q : 10
p2q : 20
p3q : 30
p4q : 40
p5q : 50
p6q : 60
p7q : 70
p8q : 80
p9q : 90


In [3]:
myquantity = 0
def getquant():
    global myquantity
    global order_type
    global symbol_ip
    allpos = kite.positions()['net']
    for i in range(len(allpos)):
        if(allpos[i]['tradingsymbol']==symbol_ip and  allpos[i]['product']== order_type):
            myquantity = allpos[i]['quantity']
            print('My Quantity : ' + str(allpos[i]['quantity']))

getquant()

def placeneworder(quantdiff):
    global stopbuy
    global stopsell
    global order_type
    global symbol_ip
    if(quantdiff>0 and stopbuy == False):
        try:
                order_id= kite.place_order(tradingsymbol=symbol_ip,
                            exchange=kite.EXCHANGE_CDS,
                            transaction_type=kite.TRANSACTION_TYPE_BUY,
                            quantity=abs(quantdiff),
                            order_type=kite.ORDER_TYPE_MARKET,
                            variety = kite.VARIETY_REGULAR,
                            product=kite.PRODUCT_MIS if order_type=='MIS' else kite.PRODUCT_NRML)
                getquant()
                stopsell = False
        except Exception as e:
            stopbuy = False
            print(e)
            getquant()
    if(quantdiff<0 and stopsell == False):
        try:
                order_id= kite.place_order(tradingsymbol=symbol_ip,
                            exchange=kite.EXCHANGE_CDS,
                            transaction_type=kite.TRANSACTION_TYPE_SELL,
                            quantity=abs(quantdiff),
                            order_type=kite.ORDER_TYPE_MARKET,
                            variety = kite.VARIETY_REGULAR,
                            product=kite.PRODUCT_MIS if order_type=='MIS' else kite.PRODUCT_NRML)
                getquant()
                stopbuy = False
        except Exception as e:
            stopsell = True
            print(e)
            getquant()


def ckqnt (orderprice,orderquant):
    global myquantity
    quantdiff = orderquant - myquantity
    placeneworder(quantdiff)
    return myquantity


last_price = pivot_price

def checkpivot(ltps):
    global last_price
    trade_quant = 0
    
    if(ltps > (globals()['p'+str(max_no_of_order)+'s']+(2*difference) ) or ltps < (globals()['p'+str(max_no_of_order)+'b']-(2*difference) )):
        ckqnt(pivot_price,0)
    
    elif ((ltps>=(pivot_price-0.0025)) and (ltps<=(pivot_price+0.0025))):
        ckqnt(pivot_price,0)
        print('in pivot range')

    else:
        for i in range(1,int(total_parts/2)+1):

            if(ltps<pivot_price):
                trade_quant = globals()['p'+str(i)+'q']
            elif(ltps>pivot_price):
                trade_quant = -globals()['p'+str(i)+'q']

            if ((ltps == globals()['p'+str(i)+'b'] or ltps == globals()['p'+str(i)+'s']) and ltps < last_price):
                ckqnt(globals()['p'+str(i)+'b'],trade_quant)
                print('==========' +'p'+str(i)+'b @ ' +str(ltps) + '==============' )
                last_price=ltps

            if ((ltps == globals()['p'+str(i)+'b'] or ltps == globals()['p'+str(i)+'s']) and ltps > last_price):
                ckqnt(globals()['p'+str(i)+'s'],trade_quant)
                print('==========' +'p'+str(i)+'s @ ' +str(ltps) + '==============' )
                last_price=ltps

My Quantity : 50


In [ ]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#              START TRADING             !
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

import logging
from kiteconnect import KiteTicker
logging.basicConfig(level=logging.DEBUG)
kws = KiteTicker("w19o0chuo929jxkp", "NzSmFguNPCcHd1nmRMIBbLIROFvXM44v")

def on_ticks(ws, ticks):
    global pivot_price
    checkpivot(ticks[0]['last_price'])

    #pivot_price =round(base * round(   ticks[0]['average_price']   /base),4)
    #calc_pivots();
    #print(hjhj)
    #print('===========================')
    #print()
    #print(type(ticks[0]['last_price']))
    print('==========================')
    print( 'LTP : ' + str(ticks[0]['last_price']))
    #print('==========================')
    #print()

def on_connect(ws, response):
    global inst_token
    ws.subscribe([inst_token])
    ws.set_mode(ws.MODE_FULL, [inst_token])

def on_close(ws, code, reason):
    ws.stop()
    
def on_error(ws, code, reason):
    logging.error("closed connection on error: {} {}".format(code, reason))

def on_noreconnect(ws):
    logging.error("Reconnecting the websocket failed")

def on_reconnect(ws, attempt_count):
    logging.debug("Reconnecting the websocket: {}".format(attempt_count))

#def on_order_update(ws, data):
#    print("order update: ", data)

kws.on_error = on_error
kws.on_noreconnect = on_noreconnect
kws.on_reconnect = on_reconnect
#kws.on_order_update = on_order_update

kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

kws.connect()

LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5225
LTP : 69.5225
LTP : 69.5225
LTP : 69.5225
LTP : 69.5225
LTP : 69.5225
LTP : 69.5225
LTP : 69.5225
LTP : 69.5225
LTP : 69.5225
LTP : 69.5225
LTP : 69.5225
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.52
LTP : 69.52
LTP : 69.52
LTP : 69.52
LTP : 69.52
LTP : 69.52
LTP : 69.52
LTP : 69.52
LTP : 69.525
LTP : 69.525
LTP : 69.525
LTP : 69.525
LTP : 69.525
LTP : 69.525
LTP : 69.525
LTP : 69.5125
LTP : 69.5125
LTP : 69.5125
LTP : 69.5125
LTP : 69.5125
LTP : 69.5125
LTP : 69.5125
LTP : 69.5125
LTP : 69.51
LTP : 69.51
LTP : 69.51
LTP : 69.5075
LTP : 69.5075
LTP : 69.5075
LTP : 69.5075
LTP : 69.5075
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 69.5175
LTP : 6

In [ ]:
order_id= kite.place_order(tradingsymbol="JPYINR20OCTFUT",
                        exchange=kite.EXCHANGE_CDS,
                        transaction_type=kite.TRANSACTION_TYPE_BUY,
                        quantity=50,
                        order_type=kite.ORDER_TYPE_MARKET,
                        variety = kite.VARIETY_AMO,
                        product=kite.PRODUCT_NRML)

In [ ]:
kitehistorical_data(instrument_token, from_date, to_date, interval, continuous=False)